In [1]:
import numpy as np
import xgboost as xgb
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error
import optuna
import pickle
from datetime import timedelta
import time
import os 
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
import gc
import tensorflow as tf

2024-05-05 17:55:45.069975: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-05 17:55:45.634096: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
feat = pd.read_csv('./data/test.csv')
FEATURE_COLS = feat.columns[1:].tolist()

In [3]:
study_name = '426_convnextbase_003_998_1'

In [4]:
pickle_file_path = f'./data/test_{study_name}.pickle'

with open(pickle_file_path, 'rb') as f:
    test_df = pickle.load(f)

pickle_file_path = f'./data/train_{study_name}.pickle'

with open(pickle_file_path, 'rb') as f:
    train_df = pickle.load(f)

In [5]:
import warnings

# Ohita tietyn tyyppiset varoitukset
warnings.filterwarnings('ignore', category=UserWarning)

def get_combined_data(df):
    # Oletetaan, että FEATURES_COLS on jo määritelty olemassa oleville piirteille
    data = [df[col].values for col in FEATURE_COLS]
    # Lisää mallin piirteet
    data.append(np.vstack(df['combined_features'].values))
    return np.column_stack(data)

def objective(trial, df, target, fold_train, fold_validation):
    param = {        
        'objective': 'reg:squarederror',        
        'device' : 'cuda',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log = True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log = True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 0.9),
        'subsample': trial.suggest_float('subsample', 0.1, 0.9),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),        
        'max_depth': trial.suggest_int('max_depth', 2, 20),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 100),
        'feature_selector': trial.suggest_categorical('feature_selector', ['shuffle', 'greedy', 'thrifty', 'cyclic', 'random']),
        'boosting': trial.suggest_categorical('boosting', ['gbtree', 'gblinear', 'dart'])

        }
    

    train_data = df[df['fold'] == fold_train]
    valid_data = df[df['fold'] == fold_validation]

    selector = trial.suggest_categorical('selector', ['f_regression', 'mutual_info_regression', 'none'])
    num_selected = trial.suggest_int('num_selected', 1, train_data.shape[1] - 1)

    if selector == 'f_regression':
        select = SelectKBest(f_regression, k=num_selected)
    elif selector == 'mutual_info_regression':
        select = SelectKBest(mutual_info_regression, k=num_selected)
    elif selector == 'none':
        select = None

    if select is not None:
        X_train = select.fit_transform(get_combined_data(train_data), train_data[target])
        X_valid = select.transform(get_combined_data(valid_data))
    else:
        X_train = get_combined_data(train_data)
        X_valid = get_combined_data(valid_data)

    y_train = train_data[target]
    y_valid = valid_data[target]

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_valid, label=y_valid)

    num_boost_round = trial.suggest_int('n_estimators', 10, 1000, log=True)
    early_stopping_rounds = trial.suggest_int('early_stopping_rounds', 5, 50)

    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    model = xgb.train(param, dtrain, num_boost_round=num_boost_round, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=False)
    preds = model.predict(dvalid)
    mse = mean_squared_error(y_valid, preds)

    return mse

def optimize_model(df, target, fold_train, fold_validation):

    if os.path.exists(f'./NN_search/{study_name}_{fold_train}_{fold_validation}_{target}_genesampler.pickle'):            
        with open(f'./NN_search/{study_name}_{fold_train}_{fold_validation}_{target}_genesampler.pickle', 'rb') as f:
            print(f'Loading gene sampler from file {f}')
            genemachine = pickle.load(f)

    else:            
        print('Creating new gene sampler')
        genemachine = optuna.samplers.NSGAIISampler(crossover = optuna.samplers.nsgaii.VSBXCrossover())

    if os.path.exists(f'./NN_search/{study_name}_{fold_train}_{fold_validation}_{target}_qmc_sampler.pickle'):
        with open(f'./NN_search/{study_name}_{fold_train}_{fold_validation}_{target}_qmc_sampler.pickle', 'rb') as f:
            print(f'Loading QMC sampler from file {f}')
            qmc_sampler = pickle.load(f)
    else:
        print(f'Creating new QMC sampler')
        qmc_sampler = optuna.samplers.QMCSampler(warn_independent_sampling = False)

    if os.path.exists(f'./NN_search/{study_name}_{fold_train}_{fold_validation}_{target}_tpe_sampler.pickle'):
        with open(f'./NN_search/{study_name}_{fold_train}_{fold_validation}_{target}_tpe_sampler.pickle', 'rb') as f:
            print(f'Loading TPE sampler from file {f}')
            tpe_sampler = pickle.load(f)
    else:
        print(f'Creating new TPE sampler')
        tpe_sampler = optuna.samplers.TPESampler(n_startup_trials=0, multivariate=True, warn_independent_sampling = False)


    start_time = time.time()
    study = optuna.create_study(direction='minimize',
                            study_name=study_name,
                            storage=f'sqlite:///502_xgboost_{target}.db',
                            load_if_exists=True                            
                            )
    
    print(f'Starting optimization for {target} with qmc sampler')
    random_time = time.time()
    study.sampler = qmc_sampler
    study.optimize(lambda trial: objective(trial, df, target, fold_train, fold_validation), n_trials=50)
    print(f'QCM optimization finished in {timedelta(seconds=time.time() - random_time)}')

    print(f'Saving QMC sampler to file ./NN_search/{study_name}_{fold_train}_{fold_validation}_{target}_qmc_sampler.pickle')
    with open(f'./NN_search/{study_name}_{fold_train}_{fold_validation}_{target}_qmc_sampler.pickle', 'wb') as f:
        pickle.dump(qmc_sampler, f)

    print(f'Starting optimization for {target} with gene sampler')
    gene_time = time.time()
    study.sampler = genemachine
    study.optimize(lambda trial: objective(trial, df, target, fold_train, fold_validation), n_trials=50)
    print(f'Gene optimization finished in {timedelta(seconds=time.time() - gene_time)}')

    print(f'Saving gene sampler to file ./NN_search/{study_name}_{fold_train}_{fold_validation}_{target}_genesampler.pickle')
    with open(f'./NN_search/{study_name}_{fold_train}_{fold_validation}_{target}_genesampler.pickle', 'wb') as f:
        pickle.dump(genemachine, f)

    print(f'Starting optimization for {target} with TPE sampler')
    tpe_time = time.time()
    study.sampler = tpe_sampler
    study.optimize(lambda trial: objective(trial, df, target, fold_train, fold_validation), n_trials=50)
    print(f'TPE optimization finished in {timedelta(seconds=time.time() - tpe_time)}')

    print(f'Saving TPE sampler to file ./NN_search/{study_name}_{fold_train}_{fold_validation}_{target}_tpe_sampler.pickle')
    with open(f'./NN_search/{study_name}_{fold_train}_{fold_validation}_{target}_tpe_sampler.pickle', 'wb') as f:
        pickle.dump(tpe_sampler, f)

    print(f'Optimization finished in {timedelta(seconds=time.time() - start_time)}')

    best_params = study.best_trial.params    

    mse_scores = []

    cross_time = time.time()

    
    tf.keras.backend.clear_session()
    gc.collect()


    for fold in df['fold'].unique():

        train_data = df[df['fold'] != fold]
        valid_data = df[df['fold'] == fold]

        selector = best_params['selector']
        num_selected = best_params['num_selected']

        if selector == 'f_regression':
            select = SelectKBest(f_regression, k=num_selected)
        elif selector == 'mutual_info_regression':
            select = SelectKBest(mutual_info_regression, k=num_selected)
        elif selector == 'none':
            select = None

        if select is not None:
            X_train = select.fit_transform(get_combined_data(train_data), train_data[target])
            X_valid = select.transform(get_combined_data(valid_data))
        
        else:
            X_train = get_combined_data(train_data)
            X_valid = get_combined_data(valid_data)


        y_train = train_data[target]
        y_valid = valid_data[target]

        dtrain = xgb.DMatrix(X_train, label=y_train)
        dvalid = xgb.DMatrix(X_valid, label=y_valid)

        

        num_boost_round = best_params['n_estimators']
        early_stopping_rounds = best_params['early_stopping_rounds']


        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        model = xgb.train(best_params, dtrain, num_boost_round=num_boost_round, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=False)
        preds = model.predict(dvalid)        
        mse = mean_squared_error(y_valid, preds)
        mse_scores.append(mse)

    average_mse = np.mean(mse_scores)

    print(f'Cross-validated MSE: {average_mse} for {target}')

    print(f"Best parameters for {target}: ", best_params)

    print(f'Train data shape without selector {train_data.shape[1]}')

    print(f'Time taken for crosvalidation: {timedelta(seconds=time.time() - cross_time)}')


    print(f'Fitting model for {target} with best parameters')

    
    tf.keras.backend.clear_session()
    gc.collect()

    del best_params['early_stopping_rounds']
   
    endfit_time = time.time()
        
    X = get_combined_data(df)
    y = df[target]

    # Käsittele featureiden valinta
    if best_params['selector'] != 'none':
        if best_params['selector'] == 'f_regression':
            select = SelectKBest(f_regression, k=best_params['num_selected'])
            num_selected = best_params['num_selected']
            print(f'Selector is f_regression with num selected {num_selected}')
        elif best_params['selector'] == 'mutual_info_regression':
            select = SelectKBest(mutual_info_regression, k=best_params['num_selected'])
            num_selected = best_params['num_selected']
            print(f'Selector is mutual_info_regression with num selected {num_selected}')
                  
        X_selected = select.fit_transform(X, y)

    else:
        X_selected = X
        print(f'None selector')

    # Luo ja kouluta lopullinen malli
    model = xgb.XGBRegressor(**best_params)
    model.fit(X_selected, y)

    print(f'Time taken for fitting whole data: {timedelta(seconds=time.time() - endfit_time)}')
    
    return model



In [6]:
train_df.head()

,id,WORLDCLIM_BIO1_annual_mean_temperature,WORLDCLIM_BIO12_annual_precipitation,WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month,WORLDCLIM_BIO15_precipitation_seasonality,WORLDCLIM_BIO4_temperature_seasonality,WORLDCLIM_BIO7_temperature_annual_range,SOIL_bdod_0.5cm_mean_0.01_deg,SOIL_bdod_100.200cm_mean_0.01_deg,SOIL_bdod_15.30cm_mean_0.01_deg,...,bin_0,bin_1,bin_2,bin_3,bin_4,bin_5,final_bin,fold,features_avg,model_features_426_convnextbase_003_998_1
0,192027691,12.235703,374.466675,62.524445,72.256844,773.592041,33.277779,125,149,136,...,2,2,1,4,2,1,221421,2.0,"[-0.14304829, -0.28004962, 0.886131, -0.183084...","[0.9996191263198853, 0.999675989151001, -0.269..."
1,195542235,17.270555,90.239998,10.351111,38.220940,859.193298,40.009777,124,144,138,...,3,3,2,2,2,3,332223,4.0,"[0.16563013, -1.4509088, 0.46862665, 0.2771467...","[0.9998477697372437, 0.9998577237129211, 0.975..."
2,196639184,14.254504,902.071411,49.642857,17.873655,387.977753,22.807142,107,133,119,...,5,1,5,5,2,3,515523,2.0,"[-0.034171782, -0.13625506, 0.5971655, -0.6490...","[0.9998796582221985, 0.9999839067459106, 0.979..."
3,195728812,18.680834,1473.933350,163.100006,45.009758,381.053986,20.436666,120,131,125,...,3,2,3,2,1,3,323213,0.0,"[-0.881118, -0.16722384, 1.3514438, 0.34779415...","[0.9997825622558594, 0.9996305108070374, 0.995..."
4,195251545,0.673204,530.088867,50.857777,38.230709,1323.526855,45.891998,91,146,120,...,2,3,3,5,4,4,233544,4.0,"[-0.3382826, 0.3086146, 0.58881557, 0.24890125...","[0.9998947978019714, 0.9997898936271667, 0.789..."


In [7]:
def prepare_features(row):
    return np.array(row[f'model_features_{study_name}'])

train_df['combined_features'] = train_df.apply(prepare_features, axis=1)
test_df['combined_features'] = test_df.apply(prepare_features, axis=1)



In [8]:
target_columns = ['X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']
train_fold = 1
validation_fold = 2

# Mallien kouluttaminen jokaiselle kohdemuuttujalle
models = {}

time_search_start = time.time()
time_taken = 0

while time_taken < 60:
    for target in target_columns:    
        print(f'\n\nOptimizing model for {target} using train fold {train_fold} and validation fold {validation_fold}\n\n')
        models[target] = optimize_model(train_df, target, train_fold, validation_fold)
        time_taken = time.time() - time_search_start
        print(f'Time taken: {timedelta(seconds=time_taken)}')   



Optimizing model for X4_mean using train fold 1 and validation fold 2


Loading gene sampler from file <_io.BufferedReader name='./NN_search/426_convnextbase_003_998_1_1_2_X4_mean_genesampler.pickle'>
Loading QMC sampler from file <_io.BufferedReader name='./NN_search/426_convnextbase_003_998_1_1_2_X4_mean_qmc_sampler.pickle'>
Loading TPE sampler from file <_io.BufferedReader name='./NN_search/426_convnextbase_003_998_1_1_2_X4_mean_tpe_sampler.pickle'>


[I 2024-05-05 17:55:48,331] Using an existing study with name '426_convnextbase_003_998_1' instead of creating a new one.


Starting optimization for X4_mean with qmc sampler


[I 2024-05-05 17:55:50,225] Trial 30 finished with value: 0.005458857455936615 and parameters: {'lambda': 0.0031622776601683764, 'alpha': 0.0316227766016838, 'colsample_bytree': 0.45000000000000007, 'subsample': 0.85, 'learning_rate': 0.071875, 'max_depth': 17, 'min_child_weight': 94, 'feature_selector': 'shuffle', 'boosting': 'gbtree', 'selector': 'none', 'num_selected': 80, 'n_estimators': 73, 'early_stopping_rounds': 42}. Best is trial 21 with value: 0.005340167057966362.
[I 2024-05-05 17:55:50,899] Trial 31 finished with value: 0.009757438862756884 and parameters: {'lambda': 0.31622776601683833, 'alpha': 3.16227766016837e-08, 'colsample_bytree': 0.65, 'subsample': 0.25, 'learning_rate': 0.319375, 'max_depth': 12, 'min_child_weight': 19, 'feature_selector': 'cyclic', 'boosting': 'gblinear', 'selector': 'none', 'num_selected': 34, 'n_estimators': 23, 'early_stopping_rounds': 30}. Best is trial 21 with value: 0.005340167057966362.
[I 2024-05-05 17:55:51,187] Trial 32 finished with val

QCM optimization finished in 0:01:08.273258
Saving QMC sampler to file ./NN_search/426_convnextbase_003_998_1_1_2_X4_mean_qmc_sampler.pickle
Starting optimization for X4_mean with gene sampler


...

QCM optimization finished in 0:00:36.921843
Saving QMC sampler to file ./NN_search/426_convnextbase_003_998_1_1_2_X3112_mean_qmc_sampler.pickle
Starting optimization for X3112_mean with gene sampler


[I 2024-05-05 18:31:30,939] Trial 10 finished with value: 3414535.5461593964 and parameters: {'lambda': 2.0695938060970025e-05, 'alpha': 0.0026870821606643056, 'colsample_bytree': 0.4719519145789307, 'subsample': 0.586288312697555, 'learning_rate': 0.2186992507164754, 'max_depth': 10, 'min_child_weight': 28, 'feature_selector': 'cyclic', 'boosting': 'dart', 'selector': 'mutual_info_regression', 'num_selected': 36, 'n_estimators': 25, 'early_stopping_rounds': 14}. Best is trial 10 with value: 3414535.5461593964.
[I 2024-05-05 18:31:31,876] Trial 11 finished with value: 5494706.401296452 and parameters: {'lambda': 0.006422704017926952, 'alpha': 8.347326274165704e-08, 'colsample_bytree': 0.2517570552102464, 'subsample': 0.42735376348664267, 'learning_rate': 0.6033776691605403, 'max_depth': 13, 'min_child_weight': 98, 'feature_selector': 'shuffle', 'boosting': 'gbtree', 'selector': 'none', 'num_selected': 117, 'n_estimators': 353, 'early_stopping_rounds': 19}. Best is trial 10 with value: 

Gene optimization finished in 0:00:52.142819
Saving gene sampler to file ./NN_search/426_convnextbase_003_998_1_1_2_X3112_mean_genesampler.pickle
Starting optimization for X3112_mean with TPE sampler


[I 2024-05-05 18:32:08,499] Trial 20 finished with value: 3089004.2594992653 and parameters: {'lambda': 0.6377858223973664, 'alpha': 0.00536181398192638, 'colsample_bytree': 0.20129206920088138, 'subsample': 0.7480050267093725, 'learning_rate': 0.2383207222228183, 'max_depth': 6, 'min_child_weight': 53, 'feature_selector': 'random', 'boosting': 'dart', 'selector': 'f_regression', 'num_selected': 29, 'n_estimators': 24, 'early_stopping_rounds': 6}. Best is trial 12 with value: 2924214.4759506476.
[I 2024-05-05 18:32:09,021] Trial 21 finished with value: 3245677.3499918208 and parameters: {'lambda': 0.11494929302107552, 'alpha': 0.03666030443247161, 'colsample_bytree': 0.18550612810307449, 'subsample': 0.8071802136424877, 'learning_rate': 0.23990503617817072, 'max_depth': 9, 'min_child_weight': 37, 'feature_selector': 'random', 'boosting': 'dart', 'selector': 'f_regression', 'num_selected': 42, 'n_estimators': 26, 'early_stopping_rounds': 15}. Best is trial 12 with value: 2924214.4759506

TPE optimization finished in 0:00:24.845976
Saving TPE sampler to file ./NN_search/426_convnextbase_003_998_1_1_2_X3112_mean_tpe_sampler.pickle
Optimization finished in 0:01:54.044254
Cross-validated MSE: 3266541.525374479 for X3112_mean
Best parameters for X3112_mean:  {'lambda': 3.380793656543689e-05, 'alpha': 6.63414198119049e-08, 'colsample_bytree': 0.7488892925355332, 'subsample': 0.5180445842959263, 'learning_rate': 0.04497247283610099, 'max_depth': 11, 'min_child_weight': 22, 'feature_selector': 'random', 'boosting': 'dart', 'selector': 'none', 'num_selected': 100, 'n_estimators': 57, 'early_stopping_rounds': 13}
Train data shape without selector 182
Time taken for crosvalidation: 0:00:38.693569
Fitting model for X3112_mean with best parameters
None selector
Time taken for fitting whole data: 0:00:08.027991
Time taken: 0:37:31.793595


In [13]:
features_array = np.array(train_df['combined_features'].tolist())
X_combined_train = np.hstack([train_df[FEATURE_COLS].values, features_array])

train_pred = np.zeros((train_df.shape[0], len(target_columns)))

selectors = {}

for i, target in enumerate(target_columns):    
    
    study = optuna.create_study(direction='minimize',
                        study_name=study_name,
                        storage=f'sqlite:///502_xgboost_{target}.db',
                        load_if_exists=True                            
                            )
    
    best_params = study.best_trial.params

    testa = best_params['selector']
    print(f'best params {testa}' )

    if best_params['selector'] != 'none':
        if best_params['selector'] == 'f_regression':
            print(f'Selector for {target}: {best_params["selector"]} with {best_params["num_selected"]} features')
            select = SelectKBest(f_regression, k=best_params['num_selected'])
            
        else:
            print(f'Selector for {target}: {best_params["selector"]} with {best_params["num_selected"]} features')
            select = SelectKBest(mutual_info_regression, k=best_params['num_selected'])
            
        X_selected = select.fit_transform(X_combined_train, train_df[target])
        selectors[target] = select

    else:
                 
        X_selected = X_combined_train
        print(f'X_selected shape {X_selected.shape}')
        print(f'None selector in target {target}')
        selectors[target] = None
    
    train_pred[:, i] = models[target].predict(X_selected)

train_r2 = r2_score(train_df[target_columns], train_pred)
print(f'Training R2: {train_r2}')

<>:21: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:21: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/tmp/ipykernel_38604/1696103066.py:21: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if best_params['selector'] is not 'none':
[I 2024-05-05 18:46:17,865] Using an existing study with name '426_convnextbase_003_998_1' instead of creating a new one.


best params none
Selector for X18_mean: none with 107 features


In [ ]:
features_array = np.array(test_df['combined_features'].tolist())
X_combined_test = np.hstack([test_df[FEATURE_COLS].values, features_array])

test_preds = np.zeros((len(test_df), len(target_columns)))

for i, target in enumerate(target_columns):
    print(f'Predicting {target} with model {models[target]}')
    study = optuna.create_study(direction='minimize',
                        study_name=study_name,
                        storage=f'sqlite:///502_xgboost_{target}_selectestii.db',
                        load_if_exists=True                            
                            )
    
    if selectors[target] is not None:
        X_selected = selectors[target].transform(X_combined_test)
    else:
        X_selected = X_combined_test

    test_preds[:, i] = models[target].predict(X_selected)
     

In [ ]:
target_columns = ['X4', 'X11', 'X18', 'X50', 'X26', 'X3112']

test_df_copy = test_df.copy()
submission_df = test_df_copy[['id']].copy()
submission_df[target_columns] = test_preds

In [ ]:
submission_df.describe()

In [ ]:
target_columns = ['X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']
train_df[target_columns].describe()

In [ ]:
submission_df.head()

In [ ]:
submission_df.to_csv('./data/submission.csv', index=False)